# Pre-Proceso (2)

Creación de nuevas variables

Definir SparkContext y SQLContext

In [1]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc)


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/22 11:13:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/22 11:13:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/22 11:13:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/02/22 11:13:29 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
/home/aldos/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
bd = sqlContext.read.format("com.databricks.spark.csv"
).option("header", "true"
).load("../On_Time_On_Time_Performance_2016_12.csv", 
       inferSchema=True)

In [3]:

bd=bd.select('Year','Month','DayofMonth','DayOfWeek','CRSDepTime','UniqueCarrier', 'TailNum', 'ArrDelay','DepDelay','Origin','Dest','Distance','Cancelled', 'Diverted','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay') 

In [4]:
bd2 = bd.filter( bd.Origin.isin('ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') & bd.Dest.isin(
    'ATL','ORD','DEN','LAX','DFW','SFO','PHX','LAS') )

sqlContext.registerDataFrameAsTable(bd2, "bd2")

22/02/22 11:14:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
bd3 = bd2.filter(bd2.Cancelled==0
).filter(bd2.Diverted==0)

In [6]:
bd4 = bd3.na.fill({'CarrierDelay':0, 'WeatherDelay':0,'NASDelay':0,'SecurityDelay':0, 'LateAircraftDelay':0})
#bd4 = bd3.na.fill(0)
#bd4 = bd3.na.drop()

## Transformación y Creación de nuevas variables

In [7]:
from pyspark.sql.functions import log

bd4 = bd4.withColumn('LogD', log(10.0, 'Distance'))

In [8]:
bd4 = bd4.withColumn('Retraso', (bd4.ArrDelay >=15).cast('int'))

In [9]:
bd4 = bd4.withColumn('RetrasoNeto', bd4.ArrDelay-bd4.DepDelay)

In [10]:
bd4.dtypes

[('Year', 'int'),
 ('Month', 'int'),
 ('DayofMonth', 'int'),
 ('DayOfWeek', 'int'),
 ('CRSDepTime', 'int'),
 ('UniqueCarrier', 'string'),
 ('TailNum', 'string'),
 ('ArrDelay', 'double'),
 ('DepDelay', 'double'),
 ('Origin', 'string'),
 ('Dest', 'string'),
 ('Distance', 'double'),
 ('Cancelled', 'double'),
 ('Diverted', 'double'),
 ('CarrierDelay', 'double'),
 ('WeatherDelay', 'double'),
 ('NASDelay', 'double'),
 ('SecurityDelay', 'double'),
 ('LateAircraftDelay', 'double'),
 ('LogD', 'double'),
 ('Retraso', 'int'),
 ('RetrasoNeto', 'double')]

In [11]:
bd4.describe('ArrDelay','LogD','Retraso','RetrasoNeto').show()

+-------+------------------+------------------+------------------+------------------+
|summary|          ArrDelay|              LogD|           Retraso|       RetrasoNeto|
+-------+------------------+------------------+------------------+------------------+
|  count|             30466|             30466|             30466|             30466|
|   mean|13.300695857677411| 2.886007454172272|0.2757500164117377|-3.919680955819602|
| stddev| 53.50946738163739|0.2817926773467498|0.4468987584631849|15.560721063721349|
|    min|             -53.0|2.3729120029701067|                 0|             -48.0|
|    max|            2028.0| 3.330210784571528|                 1|             229.0|
+-------+------------------+------------------+------------------+------------------+



In [12]:
bd4.describe('ArrDelay','LogD','Retraso','RetrasoNeto').toPandas()

summary            ArrDelay                LogD             Retraso  \
0   count               30466               30466               30466   
1    mean  13.300695857677411   2.886007454172272  0.2757500164117377   
2  stddev   53.50946738163739  0.2817926773467498  0.4468987584631849   
3     min               -53.0  2.3729120029701067                   0   
4     max              2028.0   3.330210784571528                   1   

          RetrasoNeto  
0               30466  
1  -3.919680955819602  
2  15.560721063721349  
3               -48.0  
4               229.0

In [13]:
sqlContext.registerDataFrameAsTable(bd4, "bd4")

bd5 = sqlContext.sql("select *, case \
               when CRSDepTime <= 800 then 1 \
               when 800 < CRSDepTime and CRSDepTime <= 1200 then 2 \
               when 1200 < CRSDepTime and CRSDepTime <= 1600 then 3 \
               when 1600 < CRSDepTime and CRSDepTime <= 2100 then 4 \
               else 1 end as Horario \
               from bd4")

In [14]:
bd5.select('Horario').describe().show()

+-------+-----------------+
|summary|          Horario|
+-------+-----------------+
|  count|            30466|
|   mean|2.539322523468785|
| stddev|1.136901609153113|
|    min|                1|
|    max|                4|
+-------+-----------------+



In [15]:
bd5.select('Horario').groupBy('Horario').count().show()

+-------+-----+
|Horario|count|
+-------+-----+
|      1| 7519|
|      3| 7180|
|      4| 8385|
|      2| 7382|
+-------+-----+



## Exportar la base de datos  a HDFS

In [16]:
bd5.write.format(
    "com.databricks.spark.csv"
).option(
    "header", "true"
).mode('overwrite').save('../bd5.csv') 
